In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import numpy as np
import math
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch

In [3]:
# Specify the path of your CSV file
file_path = '/kaggle/input/global-weather-repository/GlobalWeatherRepository.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to verify the reading
print(df)

           country     location_name  latitude  longitude         timezone  \
0      Afghanistan             Kabul     34.52      69.18       Asia/Kabul   
1          Albania            Tirana     41.33      19.82    Europe/Tirane   
2          Algeria           Algiers     36.76       3.05   Africa/Algiers   
3          Andorra  Andorra La Vella     42.50       1.52   Europe/Andorra   
4           Angola            Luanda     -8.84      13.23    Africa/Luanda   
...            ...               ...       ...        ...              ...   
33914    Venezuela           Caracas     10.50     -66.92  America/Caracas   
33915      Vietnam             Hanoi     21.03     105.85     Asia/Bangkok   
33916        Yemen             Sanaa     15.35      44.21        Asia/Aden   
33917       Zambia            Lusaka    -15.42      28.28    Africa/Lusaka   
33918     Zimbabwe            Harare    -17.82      31.04    Africa/Harare   

       last_updated_epoch      last_updated  temperature_celsiu

In [4]:
# Filter rows that have "Europe/Vienna" as the timezone
dfEV = df.loc[df['timezone'] == 'Europe/Vienna']

# Reset the index of the resulting DataFrame
dfEV.reset_index(drop=True, inplace=True)

# Display the resulting DataFrame
print(dfEV)

# Count the number of unique values in the 'country' column
num_unique_countries = dfEV['country'].nunique()

# Display the number of unique values
print("Number of unique values in the 'country' column:", num_unique_countries)

     country location_name  latitude  longitude       timezone  \
0    Austria        Vienna      48.2      16.37  Europe/Vienna   
1    Austria        Vienna      48.2      16.37  Europe/Vienna   
2    Austria        Vienna      48.2      16.37  Europe/Vienna   
3    Austria        Vienna      48.2      16.37  Europe/Vienna   
4    Austria        Vienna      48.2      16.37  Europe/Vienna   
..       ...           ...       ...        ...            ...   
169  Austria        Vienna      48.2      16.37  Europe/Vienna   
170  Austria        Vienna      48.2      16.37  Europe/Vienna   
171  Austria        Vienna      48.2      16.37  Europe/Vienna   
172  Austria        Vienna      48.2      16.37  Europe/Vienna   
173  Austria        Vienna      48.2      16.37  Europe/Vienna   

     last_updated_epoch      last_updated  temperature_celsius  \
0            1693301400  2023-08-29 11:30                 16.0   
1            1693363500  2023-08-30 04:45                 15.0   
2        

In [5]:
print(dfEV.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 41 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   country                       174 non-null    object 
 1   location_name                 174 non-null    object 
 2   latitude                      174 non-null    float64
 3   longitude                     174 non-null    float64
 4   timezone                      174 non-null    object 
 5   last_updated_epoch            174 non-null    int64  
 6   last_updated                  174 non-null    object 
 7   temperature_celsius           174 non-null    float64
 8   temperature_fahrenheit        174 non-null    float64
 9   condition_text                174 non-null    object 
 10  wind_mph                      174 non-null    float64
 11  wind_kph                      174 non-null    float64
 12  wind_degree                   174 non-null    int64  
 13  wind_

In [6]:
# Specify the column names to be deleted
columns_to_delete = ['country', 'location_name', 'latitude', 'longitude', 'timezone', 'last_updated_epoch',
                      'condition_text', 'wind_mph', 'wind_kph', 'wind_degree', 'pressure_mb', 'pressure_in',
                      'wind_direction', 'precip_mm', 'precip_in', 'air_quality_PM2.5', 'air_quality_PM10',
                      'visibility_km', 'air_quality_us-epa-index', 'air_quality_gb-defra-index', 'moon_illumination',
                      'moon_phase', 'moonset', 'moonrise', 'visibility_miles', 'uv_index', 'gust_mph', 'gust_kph',
                      'air_quality_Carbon_Monoxide', 'air_quality_Ozone', 'air_quality_Nitrogen_dioxide',
                      'air_quality_Sulphur_dioxide', 'cloud']

# Use the drop method to delete the specified columns
dfEV = dfEV.drop(columns=columns_to_delete)

# Split the 'last_updated' column into separate columns for year, month, and daytime
dfEV[['Year', 'Month', 'DayTime']] = dfEV['last_updated'].str.split('-', expand=True, n=2)

# Further split the 'DayTime' column into separate columns for day and time
dfEV[['Day', 'Time']] = dfEV['DayTime'].str.split(' ', expand=True, n=1)

# Convert the 'Year', 'Month', 'Day' and 'Time' columns to numeric values
dfEV['Year'] = pd.to_numeric(dfEV['Year'])
dfEV['Month'] = pd.to_numeric(dfEV['Month'])
dfEV['Day'] = pd.to_numeric(dfEV['Day'])

# Drop unnecessary columns
dfEV = dfEV.drop(['DayTime', 'Time', 'Year', 'last_updated'], axis=1)

# Display the resulting DataFrame
print(dfEV)

     temperature_celsius  temperature_fahrenheit  humidity  \
0                   16.0                    60.8        82   
1                   15.0                    59.0        82   
2                   17.0                    62.6        63   
3                   16.0                    60.8        77   
4                   18.0                    64.4        77   
..                   ...                     ...       ...   
169                 10.0                    50.0        66   
170                  8.0                    46.4        76   
171                  9.0                    48.2        62   
172                  9.0                    48.2        81   
173                 10.0                    50.0        76   

     feels_like_celsius  feels_like_fahrenheit   sunrise    sunset  Month  Day  
0                  16.0                   60.8  06:07 AM  07:43 PM      8   29  
1                  14.0                   57.1  06:09 AM  07:41 PM      8   30  
2           

In [7]:
dfIEA = pd.read_csv('/kaggle/input/iea-monthly-electricity-statistics/data.csv')

dfIEA = dfIEA[dfIEA['PRODUCT'] == 'Solar']

dfIEA = dfIEA.drop('PRODUCT', axis=1)
dfIEA = dfIEA.drop('MONTH_NAME', axis=1)
dfIEA = dfIEA.drop('TIME', axis=1)
dfIEA = dfIEA.drop('CODE_TIME', axis=1)
dfIEA = dfIEA.drop('DISPLAY_ORDER', axis=1)
dfIEA = dfIEA.drop('YEAR', axis=1)

dfIEA = dfIEA.loc[dfIEA['COUNTRY'] == 'Austria']


dfIEA.reset_index(drop=True, inplace=True)

print(dfIEA)

     COUNTRY  MONTH       VALUE  yearToDate  previousYearToDate     share
0    Austria      1    2.864000      84.421                 NaN  0.000485
1    Austria      2    4.692000      84.421                 NaN  0.000885
2    Austria      3    7.925000      84.421                 NaN  0.001424
3    Austria      4    9.965000      84.421                 NaN  0.002280
4    Austria      5   10.200000      84.421                 NaN  0.001849
..       ...    ...         ...         ...                 ...       ...
151  Austria      8  302.743959    2677.672         1510.816132  0.067383
152  Austria      9  207.410436    2677.672         1690.628660  0.041086
153  Austria     10  160.138077    2677.672         1827.442540  0.031323
154  Austria     11   85.906672    2677.672         1899.758448  0.017416
155  Austria     12   65.434666    2677.672         1956.438484  0.012272

[156 rows x 6 columns]


In [8]:
# Convert column names to lowercase for both DataFrames
dfIEA.columns = dfIEA.columns.str.lower()
dfEV.columns = dfEV.columns.str.lower()

# Merge the two DataFrames based on 'year' and 'month'
merged_df = pd.merge(dfIEA, dfEV, left_on=['month'], right_on=['month'], how='inner')

# Display the resulting DataFrame
print(merged_df)

      country  month      value  yeartodate  previousyeartodate     share  \
0     Austria      1   2.864000      84.421                 NaN  0.000485   
1     Austria      1   2.864000      84.421                 NaN  0.000485   
2     Austria      1   2.864000      84.421                 NaN  0.000485   
3     Austria      1   2.864000      84.421                 NaN  0.000485   
4     Austria      1   2.864000      84.421                 NaN  0.000485   
...       ...    ...        ...         ...                 ...       ...   
2257  Austria     12  65.434666    2677.672         1956.438484  0.012272   
2258  Austria     12  65.434666    2677.672         1956.438484  0.012272   
2259  Austria     12  65.434666    2677.672         1956.438484  0.012272   
2260  Austria     12  65.434666    2677.672         1956.438484  0.012272   
2261  Austria     12  65.434666    2677.672         1956.438484  0.012272   

      temperature_celsius  temperature_fahrenheit  humidity  \
0           

In [9]:
merged_df = merged_df.drop('country', axis=1)

In [10]:
print(merged_df)

      month      value  yeartodate  previousyeartodate     share  \
0         1   2.864000      84.421                 NaN  0.000485   
1         1   2.864000      84.421                 NaN  0.000485   
2         1   2.864000      84.421                 NaN  0.000485   
3         1   2.864000      84.421                 NaN  0.000485   
4         1   2.864000      84.421                 NaN  0.000485   
...     ...        ...         ...                 ...       ...   
2257     12  65.434666    2677.672         1956.438484  0.012272   
2258     12  65.434666    2677.672         1956.438484  0.012272   
2259     12  65.434666    2677.672         1956.438484  0.012272   
2260     12  65.434666    2677.672         1956.438484  0.012272   
2261     12  65.434666    2677.672         1956.438484  0.012272   

      temperature_celsius  temperature_fahrenheit  humidity  \
0                     5.5                    42.0        85   
1                     4.0                    39.2        

In [11]:
# Calculate the mean of the 'previousyeartodate' column
mean_previousyeartodate = merged_df['previousyeartodate'].mean()

# Fill null values in the 'previousyeartodate' column with the calculated mean
merged_df['previousyeartodate'] = merged_df['previousyeartodate'].fillna(mean_previousyeartodate)

# Display the resulting DataFrame
print(merged_df)

print(merged_df.info())

      month      value  yeartodate  previousyeartodate     share  \
0         1   2.864000      84.421          663.638505  0.000485   
1         1   2.864000      84.421          663.638505  0.000485   
2         1   2.864000      84.421          663.638505  0.000485   
3         1   2.864000      84.421          663.638505  0.000485   
4         1   2.864000      84.421          663.638505  0.000485   
...     ...        ...         ...                 ...       ...   
2257     12  65.434666    2677.672         1956.438484  0.012272   
2258     12  65.434666    2677.672         1956.438484  0.012272   
2259     12  65.434666    2677.672         1956.438484  0.012272   
2260     12  65.434666    2677.672         1956.438484  0.012272   
2261     12  65.434666    2677.672         1956.438484  0.012272   

      temperature_celsius  temperature_fahrenheit  humidity  \
0                     5.5                    42.0        85   
1                     4.0                    39.2        

In [12]:
# Convert the 'sunrise' and 'sunset' columns to datetime
merged_df['sunrise'] = pd.to_datetime(merged_df['sunrise'], format='%I:%M %p')
merged_df['sunset'] = pd.to_datetime(merged_df['sunset'], format='%I:%M %p')

# Extract the hour from 'sunrise' and 'sunset'
merged_df['sunrise'] = merged_df['sunrise'].dt.hour
merged_df['sunset'] = merged_df['sunset'].dt.hour

# Apply one-hot encoding to the 'sunrise' and 'sunset' columns
merged_df = pd.get_dummies(merged_df, columns=['sunrise', 'sunset'], prefix=['sunrise', 'sunset'])

# Convert True/False to 1/0 in the columns sunrise_6, sunrise_7, sunset_16, sunset_17, sunset_18, sunset_19
merged_df[['sunrise_6', 'sunrise_7', 'sunset_16', 'sunset_17', 'sunset_18', 'sunset_19']] = merged_df[['sunrise_6', 'sunrise_7', 'sunset_16', 'sunset_17', 'sunset_18', 'sunset_19']].astype(int)

In [13]:
print(merged_df)

      month      value  yeartodate  previousyeartodate     share  \
0         1   2.864000      84.421          663.638505  0.000485   
1         1   2.864000      84.421          663.638505  0.000485   
2         1   2.864000      84.421          663.638505  0.000485   
3         1   2.864000      84.421          663.638505  0.000485   
4         1   2.864000      84.421          663.638505  0.000485   
...     ...        ...         ...                 ...       ...   
2257     12  65.434666    2677.672         1956.438484  0.012272   
2258     12  65.434666    2677.672         1956.438484  0.012272   
2259     12  65.434666    2677.672         1956.438484  0.012272   
2260     12  65.434666    2677.672         1956.438484  0.012272   
2261     12  65.434666    2677.672         1956.438484  0.012272   

      temperature_celsius  temperature_fahrenheit  humidity  \
0                     5.5                    42.0        85   
1                     4.0                    39.2        

In [14]:
from sklearn.preprocessing import StandardScaler

# Select the columns to normalize
columns_to_normalize = ['value', 'yeartodate', 'previousyeartodate', 'share', 'temperature_celsius', 'temperature_fahrenheit', 'humidity', 'feels_like_celsius', 'feels_like_fahrenheit']

# Create a scaler
scaler = StandardScaler()

# Normalize the selected columns
merged_df[columns_to_normalize] = scaler.fit_transform(merged_df[columns_to_normalize])

In [15]:
print(merged_df)

      month     value  yeartodate  previousyeartodate     share  \
0         1 -1.159443   -1.397166       -1.861803e-16 -1.118550   
1         1 -1.159443   -1.397166       -1.861803e-16 -1.118550   
2         1 -1.159443   -1.397166       -1.861803e-16 -1.118550   
3         1 -1.159443   -1.397166       -1.861803e-16 -1.118550   
4         1 -1.159443   -1.397166       -1.861803e-16 -1.118550   
...     ...       ...         ...                 ...       ...   
2257     12  0.113943    2.073550        2.117166e+00  0.108615   
2258     12  0.113943    2.073550        2.117166e+00  0.108615   
2259     12  0.113943    2.073550        2.117166e+00  0.108615   
2260     12  0.113943    2.073550        2.117166e+00  0.108615   
2261     12  0.113943    2.073550        2.117166e+00  0.108615   

      temperature_celsius  temperature_fahrenheit  humidity  \
0               -0.540982               -0.532523  0.828528   
1               -0.768424               -0.768382  1.512174   
2     

In [16]:
class RNNRegressionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RNNRegressionModel, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.rnn(x)
        if len(out.shape) > 2:
            out = self.fc(out[:, -1, :])
        else:
            out = self.fc(out)
        return out.squeeze(1) if len(out.shape) > 2 else out

In [17]:
import wandb

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Label")

wandb.login(key=secret_value_0)

wandb.init(project='IEA + GWW Model', save_code=True)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ales-2000-09. Use `wandb login --relogin` to force relogin


In [18]:
target_column = 'value'

X = merged_df.iloc[:, [0] + list(range(2, len(merged_df.columns)))]

y = merged_df[target_column]

# Split the dataset into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Number of input and output features
input_dim = X_train.shape[1]
output_dim = 1

# Initialize the model
#model = LinearRegressionModel(input_dim, output_dim)

# # Default Initialization
# nn.init.xavier_uniform_(model.linear.weight)
# nn.init.zeros_(model.linear.bias)

model = RNNRegressionModel(input_dim, input_dim*2, output_dim)


# Define the loss function and optimizer
criterion = torch.nn.MSELoss()
lr=0.00001
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
wandb.log({'LR': lr})

print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2262 entries, 0 to 2261
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   month                   2262 non-null   int64  
 1   value                   2262 non-null   float64
 2   yeartodate              2262 non-null   float64
 3   previousyeartodate      2262 non-null   float64
 4   share                   2262 non-null   float64
 5   temperature_celsius     2262 non-null   float64
 6   temperature_fahrenheit  2262 non-null   float64
 7   humidity                2262 non-null   float64
 8   feels_like_celsius      2262 non-null   float64
 9   feels_like_fahrenheit   2262 non-null   float64
 10  day                     2262 non-null   int64  
 11  sunrise_6               2262 non-null   int64  
 12  sunrise_7               2262 non-null   int64  
 13  sunset_16               2262 non-null   int64  
 14  sunset_17               2262 non-null   

In [19]:
# Training
epochs = 128
batch_size = 1
wandb.log({'Epoch': epochs})
wandb.log({'batch_size': batch_size})

for epoch in range(epochs):
    model.train()
    for i in range(0, X_train.shape[0], batch_size):
        inputs = torch.from_numpy(X_train.iloc[i:i+batch_size].values).float()
        labels = torch.from_numpy(y_train.iloc[i:i+batch_size].values).float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_loss_total = 0.0

    with torch.no_grad():
        for i in range(0, X_val.shape[0], batch_size):
            val_batch_inputs = torch.from_numpy(X_val.iloc[i:i+batch_size].values).float()
            val_batch_labels = torch.from_numpy(y_val.iloc[i:i+batch_size].values).float().unsqueeze(1)

            val_outputs = model(val_batch_inputs)
            val_loss = criterion(val_outputs, val_batch_labels)
            val_loss_total += val_loss.item()

    avg_val_loss = val_loss_total / (X_val.shape[0] / batch_size)

    print(f'Epoch {epoch+1}/{epochs}, Training Loss: {loss.item()}, Validation Loss: {avg_val_loss}')
    wandb.log({'Training Loss': loss.item(), 'Validation Loss': avg_val_loss})


# Test
model.eval()
with torch.no_grad():
    test_inputs = torch.from_numpy(X_test.values).float()
    test_labels = torch.from_numpy(y_test.values).float().unsqueeze(1)
    test_outputs = model(test_inputs)
    test_loss = criterion(test_outputs, test_labels)

print(f'Test Loss: {test_loss.item()}')
wandb.log({'Test Loss': test_loss.item()})

Epoch 1/128, Training Loss: 0.8960444927215576, Validation Loss: 0.9406816494208851
Epoch 2/128, Training Loss: 0.5369085669517517, Validation Loss: 0.8329397028905303
Epoch 3/128, Training Loss: 0.44737714529037476, Validation Loss: 0.7870156329882831
Epoch 4/128, Training Loss: 0.42204076051712036, Validation Loss: 0.7515826120566854
Epoch 5/128, Training Loss: 0.4119444191455841, Validation Loss: 0.7186249381036406
Epoch 6/128, Training Loss: 0.4031543731689453, Validation Loss: 0.6863566293518689
Epoch 7/128, Training Loss: 0.39226844906806946, Validation Loss: 0.6545121080472457
Epoch 8/128, Training Loss: 0.3791106343269348, Validation Loss: 0.6231838028038309
Epoch 9/128, Training Loss: 0.36409932374954224, Validation Loss: 0.5924646910743062
Epoch 10/128, Training Loss: 0.3474794030189514, Validation Loss: 0.5623860097717158
Epoch 11/128, Training Loss: 0.32931584119796753, Validation Loss: 0.5329386890228154
Epoch 12/128, Training Loss: 0.3096385896205902, Validation Loss: 0.5

In [20]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error


# Calcola le metriche di test
y_test_true = y_test.values
y_test_pred = model(torch.from_numpy(X_test.values).float()).detach().numpy()

# Calcola e stampa le metriche di test
r2_test = r2_score(y_test_true, y_test_pred)
mse_test = mean_squared_error(y_test_true, y_test_pred)
mae_test = mean_absolute_error(y_test_true, y_test_pred)

wandb.log({'r2_test': r2_test, 'mse_test': mse_test, 'mae_test':mae_test})

print(f'R2 Score: {r2_test}, Mean Squared Error: {mse_test}, Mean Absolute Error: {mae_test}')
wandb.finish()

R2 Score: 0.9135166954729765, Mean Squared Error: 0.08139250993211926, Mean Absolute Error: 0.22941606556120853


wandb: WARNING No program path found when generating artifact job source for a non-colab notebook run. See https://docs.wandb.ai/guides/launch/create-job
wandb: WARNING Source type is set to 'artifact' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Epoch,▁
LR,▁
Test Loss,▁
Training Loss,█▇▆▆▄▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Loss,█▇▆▆▅▄▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch_size,▁
mae_test,▁
mse_test,▁
r2_test,▁
Epoch,128
LR,1e-05
